In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold
import lightgbm as lgbm# .loc[df_test.sku_name == new_id_test[0]]
from sklearn.preprocessing import LabelEncoder as LE
import seaborn as sns
import scipy.stats as ss
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import seaborn as sns
from matplotlib import pyplot as plt
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:10].index
    best_features = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:50]
    best_features.reset_index(inplace=True)
    print(best_features.dtypes)
    plt.figure(figsize=(8, 10))
    sns.barplot(x="Importance", y="Feature", data=best_features)
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')
dict_s = pd.read_csv('DataDictionary.csv')


In [3]:
sellin = ['sellin','sellin_channel_1','sellin_channel_2','sellin_channel_3','sellin_channel_4',
         'sellin_channel_5','sellin_channel_6','sellin_channel_7','sellin_channel_8']
sellout = ['sellout','sellout_channel_1','sellout_channel_2','sellout_channel_3','sellout_channel_4',
         'sellout_channel_5','sellout_channel_6','sellout_channel_7','sellout_channel_8','sellout_channel_9',
           'sellout_channel_10']
onhand= ['onhand_inventory','onhand_inventory_channel_1','onhand_inventory_channel_2','onhand_inventory_channel_3','onhand_inventory_channel_4',
         'onhand_inventory_channel_5','onhand_inventory_channel_6','onhand_inventory_channel_7','onhand_inventory_channel_8','onhand_inventory_channel_9',
           'onhand_inventory_channel_10']

In [7]:
# test
import calendar
def count_weekends(m,y):
    day_to_count = calendar.SUNDAY

    matrix = calendar.monthcalendar(y,m)

    num_days = sum(1 for x in matrix if x[day_to_count] != 0)
    return num_days%4
test['Weeks'] = test.apply(lambda x: count_weekends(x.month, x.year), axis=1)
train['date'] = (train['year']-2016)*12 + (train['month']-1) 
test['date'] = (test['year']-2016)*12 + (test['month']-1) # train

In [8]:
df_train = train.copy()
df_test = test.copy()
df_train[sellin+sellout+onhand+['leftover_inventory','starting_inventory']] = df_train[sellin+sellout+onhand+['leftover_inventory','starting_inventory']]/1013

In [9]:
# df_train.columns
pcl = df_train.groupby(['sku_name','year','month'])['product_lifecycle_stage'].apply(lambda x : x.unique()[0]).reset_index()
pcl_test = df_train.groupby('sku_name')['product_lifecycle_stage'].apply(lambda x:x.values[-1])
df_train.drop('product_lifecycle_stage',axis = 1,inplace = True)
fea1 = sellin+sellout+onhand+['leftover_inventory','starting_inventory']
fea2 = [f for f in df_train.columns.tolist() if f not in fea1]
# new_target = df_train.groupby(['sku_name','year','month'])['sellin'].sum().reset_index()
new_target = df_train.groupby(['sku_name','year','month'])[fea1].sum().reset_index()
new_target2 = df_train[fea2].groupby(['sku_name','year','month']).mean().reset_index()
df_train = new_target.merge(pcl,on = ['sku_name','year','month'],how = 'left').merge(new_target2,on = ['sku_name','year','month'],how = 'left')
df_test['product_lifecycle_stage'] = df_test['sku_name'].map(pcl_test)
price = df_train.groupby(['sku_name','year','month'])['price'].apply(lambda x : x.unique()[0]).reset_index()
price_test = df_train.groupby('sku_name')['price'].apply(lambda x:x.values[-1])
# df_test['price'] = df_test['sku_name'].map(price_test)

target = 'sellin'
prediction_columns = df_test.columns.tolist()
encoding_columns = [t for t in df_train.columns if t not in prediction_columns]
mao = df_train.groupby('sku_name').size()
df_test['size'] = df_test['sku_name'].map(mao)
df_test['size'].fillna(0,inplace = True)
df_train['size'] = df_train['sku_name'].map(mao)
df_train['sellin_old'] = df_train.groupby('sku_name')[['sellin']].shift(1).fillna(0)
df_train['sellin_old2'] = df_train.groupby('sku_name')[['sellin']].shift(2).fillna(0)
df_train['period_old'] = df_train.groupby(['sku_name'])['date'].diff().fillna(0)
df_train['period_old2'] = df_train.groupby(['sku_name'])['date'].diff(2)
ma_sell = df_train[df_train.sku_name.isin(df_test.sku_name)].groupby('sku_name').apply(lambda x : x['sellin'].values[-1])
ma_date = df_train[df_train.sku_name.isin(df_test.sku_name)].groupby('sku_name').apply(lambda x : x['date'].values[-1])

ma_sell2 = df_train.loc[(df_train.sku_name.isin(df_test.sku_name))& (df_train['size']>1)].groupby('sku_name').apply(lambda x : x['sellin'].values[-2])
ma_date2 = df_train.loc[(df_train.sku_name.isin(df_test.sku_name))& (df_train['size']>1)].groupby('sku_name').apply(lambda x : x['date'].values[-2])
df_test['sellin_old'] = df_test['sku_name'].map(ma_sell)
df_test['period_old'] = df_test['sku_name'].map(ma_date)
df_test['sellin_old2'] = df_test['sku_name'].map(ma_sell2)
df_test['period_old2'] = df_test['sku_name'].map(ma_date2)
df_test['period_old'] = df_test['date'] - df_test['period_old']
df_test['period_old2'] = df_test['date'] - df_test['period_old2']


In [10]:
df_train['medians'] = df_train['sku_name'].map(df_train.groupby('sku_name')['sellin'].median())

In [11]:
def rmspe(y_true, y_pred):
#     y_true_rescaled = sca.inverse_transform(y_true.values.reshape(-1,1))
#     y_pred_rescaled = sca.inverse_transform(y_pred.reshape(-1,1))
    return  np.mean(np.abs(y_true - y_pred))*1013#

In [12]:
mask1 = df_train.sku_name.isin(df_test.sku_name)

In [15]:
# rmspe(df_train[mask1]['sellin'],df_train[mask1]['medians'])
df_train['pc'] = df_train.groupby('sku_name')['sellin'].pct_change().fillna(0)

In [32]:
# sns.distplot(df_train.pc)
# df_train.corr()['pc'].sort_values(ascending = False,key = abs)
# df_train['sellin'].cumprod()
df_train['cum_pc'] = df_train.groupby('sku_name').apply(lambda x : (x['pc']+1).cumprod()).values

In [33]:
# df_train['cum_pc']


(28023,)